In [3]:
!pip install segmentation-models-pytorch --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.3 MB/s eta 0:00:00


In [16]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
import torchvision.transforms as T
from torchvision.datasets import VOCSegmentation
import segmentation_models_pytorch as smp  # библиотека теперь установлена
from tqdm import notebook

print("Запущена Ячейка 1: проверка окружения и импорт библиотек")
print("Device:", 'CUDA' if torch.cuda.is_available() else 'CPU')
print("Torch version:", torch.__version__)

Запущена Ячейка 1: проверка окружения и импорт библиотек
Device: CUDA
Torch version: 2.6.0+cu124


In [17]:
DATA_DIR = '/content/VOC2012'

# Трансформации для изображений и масок
transform_img = T.Compose([
    T.Resize((128,128)),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
transform_mask = T.Compose([
    T.Resize((128,128)),
    T.ToTensor()
])

# Загрузка датасета и создание поднабора
full_dataset = VOCSegmentation(
    root=DATA_DIR, year='2012', image_set='train', download=True,
    transform=transform_img, target_transform=transform_mask
)
subset_size = 1000
indices = list(range(subset_size))
train_size = int(0.8 * subset_size)
val_size = subset_size - train_size
subset = torch.utils.data.Subset(full_dataset, indices)
train_ds, val_ds = random_split(subset, [train_size, val_size])

# DataLoader
train_loader = DataLoader(train_ds, batch_size=8, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=8, shuffle=False)

print("Ячейка 2: данные загружены и подготовлены")

Ячейка 2: данные загружены и подготовлены


In [18]:
 import torch
import torch.nn as nn
import segmentation_models_pytorch as smp  # импорт библиотеки
from sklearn.metrics import jaccard_score

# Определение устройства
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Функция потерь: Dice + CrossEntropy
loss_fn = lambda preds, masks: smp.losses.DiceLoss(mode='multiclass')(preds, masks) + nn.CrossEntropyLoss()(preds, masks)

# Функция обучения за одну эпоху
def train_epoch(model, loader, optimizer):
    model.train()
    total_loss = 0
    for imgs, masks in loader:
        imgs, masks = imgs.to(DEVICE), masks.squeeze(1).long().to(DEVICE)
        preds = model(imgs)
        loss = loss_fn(preds, masks)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# Функция валидации: вычисление IoU через jaccard_score
def eval_model(model, loader):
    model.eval()
    iou_scores = []
    with torch.no_grad():
        for imgs, masks in loader:
            imgs, masks = imgs.to(DEVICE), masks.squeeze(1).long().to(DEVICE)
            preds = model(imgs)
            pred_labels = preds.argmax(dim=1).cpu().numpy().flatten()
            true_labels = masks.cpu().numpy().flatten()
            iou = jaccard_score(true_labels, pred_labels, average='macro')
            iou_scores.append(iou)
    return sum(iou_scores) / len(iou_scores)

# Бейслайн модели: U-Net и SegFormer
models_baseline = {
    'unet': smp.Unet(encoder_name='resnet34', encoder_weights='imagenet', classes=21, activation=None),
    'segformer': smp.Segformer(encoder_name='mit_b0', encoder_weights='imagenet', classes=21, activation=None)
}

print("Ячейка 3: функции тренировки и модели определены")

Ячейка 3: функции тренировки и модели определены


In [23]:
from tqdm.notebook import tqdm

history = {}
for name, model in models_baseline.items():
    model.to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    # Scheduler снижает LR при остановке роста метрики
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=1, verbose=True)
    print(f"=== Тренировка {name} (бейслайн) ===")
    # Пять эпох обучения
    for epoch in range(1, 6):
        print(f"Эпоха {epoch}/5")
        epoch_loss = 0.0
        # Проход по обучающим батчам с прогресс-баром
        for imgs, masks in tqdm(train_loader, desc="Обучающие батчи", leave=False):
            imgs, masks = imgs.to(DEVICE), masks.squeeze(1).long().to(DEVICE)
            preds = model(imgs)
            loss = loss_fn(preds, masks)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        avg_loss = epoch_loss / len(train_loader)
        val_iou = eval_model(model, val_loader)
        print(f"Средняя потеря: {avg_loss:.4f}, Валидационная IoU: {val_iou:.4f}")
        scheduler.step(val_iou)
    history[name] = val_iou

print("=== Результаты бейзлайна ===")
for nm, iou_val in history.items():
    print(f"{nm}: IoU={iou_val:.4f}")
print("Ячейка 4 выполнена")

=== Тренировка unet (бейслайн) ===
Эпоха 1/5


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Обучающие батчи:   0%|          | 0/100 [00:00<?, ?it/s]

Средняя потеря: 0.2353, Валидационная IoU: 0.4770
Эпоха 2/5


Обучающие батчи:   0%|          | 0/100 [00:00<?, ?it/s]

...

Средняя потеря: 0.2197, Валидационная IoU: 0.4772
Эпоха 4/5


Обучающие батчи:   0%|          | 0/100 [00:00<?, ?it/s]

Средняя потеря: 0.2183, Валидационная IoU: 0.4770
Эпоха 5/5


Обучающие батчи:   0%|          | 0/100 [00:00<?, ?it/s]

Средняя потеря: 0.2385, Валидационная IoU: 0.4770
=== Результаты бейзлайна ===
unet: IoU=0.4771
segformer: IoU=0.4770
Ячейка 4 выполнена


In [25]:
from tqdm.notebook import tqdm as tq
# Аугментации: отражение, поворот и сдвиг
aug_transform_img = T.Compose([
    T.Resize((128,128)),
    T.RandomHorizontalFlip(),
    T.RandomRotation(15),
    T.RandomAffine(0, translate=(0.1,0.1)),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])
train_aug_ds = VOCSegmentation(
    root=DATA_DIR, year='2012', image_set='train', download=False,
    transform=aug_transform_img, target_transform=transform_mask
)
train_aug_small = torch.utils.data.Subset(train_aug_ds, indices)
train_loader_aug = DataLoader(train_aug_small, batch_size=8, shuffle=True)

# Создание модели и оптимизатора
improved_model = smp.Unet('resnet34', encoder_weights='imagenet', classes=21, activation=None).to(DEVICE)
optimizer_imp = torch.optim.Adam(improved_model.parameters(), lr=1e-4)
# Scheduler для улучшенного бейзлайна
scheduler_imp = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_imp, mode='max', factor=0.5, patience=1, verbose=True)

print("=== Тренировка улучшенного U-Net ===")
# Семь эпох обучения
for epoch in tq(range(1, 8), desc="Improved Epochs", leave=False):
    print(f"Эпоха {epoch}/7")
    batch_losses = []
    # Проход по батчам
    for imgs, masks in tq(train_loader_aug, desc="Обучающие батчи", leave=False):
        imgs, masks = imgs.to(DEVICE), masks.squeeze(1).long().to(DEVICE)
        preds = improved_model(imgs)
        loss_imp = loss_fn(preds, masks)
        optimizer_imp.zero_grad()
        loss_imp.backward()
        optimizer_imp.step()
        batch_losses.append(loss_imp.item())
    avg_loss_imp = sum(batch_losses) / len(batch_losses)
    val_iou_imp = eval_model(improved_model, val_loader)
    print(f"Средняя потеря: {avg_loss_imp:.4f}, Валидационная IoU: {val_iou_imp:.4f}")
    # Шаг scheduler
    scheduler_imp.step(val_iou_imp)

print(f"=== Улучшенный U-Net IoU={val_iou_imp:.4f} ===")
print("Ячейка 5 выполнена")

=== Тренировка улучшенного U-Net ===


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Improved Epochs:   0%|          | 0/7 [00:00<?, ?it/s]

Эпоха 1/7


Обучающие батчи:   0%|          | 0/125 [00:00<?, ?it/s]

...

Средняя потеря: 0.2414, Валидационная IoU: 0.4770
Эпоха 6/7


Обучающие батчи:   0%|          | 0/125 [00:00<?, ?it/s]

Средняя потеря: 0.2400, Валидационная IoU: 0.4770
Эпоха 7/7


Обучающие батчи:   0%|          | 0/125 [00:00<?, ?it/s]

Средняя потеря: 0.2401, Валидационная IoU: 0.4770
=== Улучшенный U-Net IoU=0.4770 ===
Ячейка 5 выполнена


In [28]:
print("Сравнение результатов:")
print(f"Baseline U-Net IoU = {history['unet']:.4f}")
print(f"Improved U-Net IoU = {val_iou_imp:.4f}")
# Вывод о подтверждении гипотез


Сравнение результатов:
Baseline U-Net IoU = 0.4771
Improved U-Net IoU = 0.4770


baseline и улучшенный U-Net показывают почти одинаковый результат (~0.47 IoU)
Скорее всего, это вызвано тем, что было проведено слишком малое количество эпох, однако в виду ограниченности времени и работы Google Collab нет возможности провести работу на большем количестве эпох. Также это может быть вызвано слишком грубой аугментацией.